# Static Yaw Misalignment MVP


## WeDoWind Rapperswil 2024-06-18

This notebook is designed to show a simple static yaw misalignment analysis using a single year of the data from Kelmarsh wind farm

## Collaborative coding

Feel free to edit and change the code, but this will change it for everyone! :D

Please add commentary where appropriate, and propose changes and improvements for others to perform.

## License

I've set the repository up on Github with an open MIT license, so anything you contribute will be open source. Hope thats okay :)

## Improvements

- Functionise the code
- Expand to more turbines
- Include the yaw misalginment frequency distribution
- Improve the power curve filtering
- Improve the comments and Markdown
- Propose your own improvements
- You can also comment with "TODO - this will make things better" within the code

In [ ]:
import requests
from pathlib import Path
from tqdm import tqdm
import logging
import re
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import zipfile


In [ ]:
logger = logging.getLogger()

In [ ]:
BYTES_MB = 1024 * 1024

In [ ]:

def download_file(url: str, outfile: str | Path) -> None:
    """
    Download a file from the web, based on its url, and save to the outfile.

    Args:
        url(:obj:`str`): Url of data to download.
        outfile(:obj:`str` | :obj:`Path`): File path to which the download is saved.
    """

    outfile = Path(outfile).resolve()
    result = requests.get(url, stream=True)

    with outfile.open("wb") as f:
        for chunk in tqdm(result.iter_content(chunk_size=BYTES_MB), desc="MB downloaded"):
            if chunk:
                f.write(chunk)


In [ ]:
url = r"https://zenodo.org/records/8252025/files/Kelmarsh_12.3MW_6xSenvion_MM92.kmz?download=1"
outfile = Path("Kelmarsh_12.3MW_6xSenvion_MM92.kmz")

if not outfile.is_file():
    download_file(url,outfile)


In [ ]:
url = r"https://zenodo.org/records/8252025/files/Kelmarsh_SCADA_2022_4457.zip?download=1"
outpath = Path("data")
outfile = outpath/Path("Kelmarsh_SCADA_2022_4457.zip")

outpath.mkdir(parents=True, exist_ok=True)

if not outfile.is_file():   
    download_file(url,outfile)

    with zipfile.ZipFile(outfile,"r") as zip_ref:
        zip_ref.extractall(outpath)


In [ ]:
scada_files = list(Path().rglob("data/Turbine_Data*.csv"))

In [ ]:
use_columns = [
            "# Date and time",
            "Power (kW)",
            "Wind speed (m/s)",
            "Wind direction (°)",
            "Nacelle position (°)",
            "Blade angle (pitch position) A (°)",
        ]

csv_params = {
        "index_col": "# Date and time",
        "parse_dates": True,
        "skiprows": 9,
        "usecols": use_columns,
    }

In [ ]:
str(scada_files[0])[-42:-32]

In [ ]:
scada_lst = []

for file in scada_files:
    turbine_name = str(file)[-42:-32] # TODO - should take name of file not the full path

    scada_wt = pd.read_csv(file, **csv_params)

    scada_wt["Turbine"] = turbine_name
    scada_wt.index.names = ["Timestamp"]
    scada_lst.append(scada_wt.copy())

scada = pd.concat(scada_lst)

In [ ]:
scada

In [ ]:
scada["Yaw error"] = scada["Nacelle position (°)"] - scada["Wind direction (°)"] # TODO - circular difference rather than absolute

In [ ]:
kelmarsh_1 = scada[scada["Turbine"]=="Kelmarsh_1"].drop(columns="Turbine")

In [ ]:
kelmarsh_1

In [ ]:
power_curve = plt.figure()
plt.plot(kelmarsh_1["Wind speed (m/s)"],kelmarsh_1["Power (kW)"],marker='.',linestyle='')

pitch_curve = plt.figure()
plt.plot(kelmarsh_1["Wind speed (m/s)"],kelmarsh_1["Blade angle (pitch position) A (°)"],marker='.',linestyle='')


In [ ]:
kelmarsh_1_filtered = kelmarsh_1[(kelmarsh_1["Blade angle (pitch position) A (°)"]<1.5) 
                                 & (kelmarsh_1["Blade angle (pitch position) A (°)"]>-1.5)]

In [ ]:
plt.figure(power_curve)
plt.plot(kelmarsh_1_filtered["Wind speed (m/s)"],kelmarsh_1_filtered["Power (kW)"],marker='.',linestyle='')
plt.show()

In [ ]:
plt.figure(pitch_curve)
plt.plot(kelmarsh_1_filtered["Wind speed (m/s)"],kelmarsh_1_filtered["Blade angle (pitch position) A (°)"],marker='.',linestyle='')
plt.show()

In [ ]:
kelmarsh_1_filtered

## A crude static yaw misalignment analysis

In [ ]:
ws_bins=[5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
vane_bins=np.linspace(-25,25,51)

kelmarsh_1_filtered['ws_bin'] = pd.cut(kelmarsh_1_filtered['Wind speed (m/s)'], ws_bins)
kelmarsh_1_filtered['yaw_bin'] = pd.cut(kelmarsh_1_filtered['Yaw error'], vane_bins)

In [ ]:
kelmarsh_1_filtered

In [ ]:
kelmarsh_1_sye_data = kelmarsh_1_filtered.groupby(by=["ws_bin","yaw_bin"])["Power (kW)"].median()/kelmarsh_1_filtered.groupby(by=["ws_bin"])["Power (kW)"].median()
kelmarsh_1_sye_data = kelmarsh_1_sye_data.reset_index()


In [ ]:
kelmarsh_1_sye_data["ws_bin_left"] = pd.IntervalIndex(kelmarsh_1_sye_data['ws_bin']).left
kelmarsh_1_sye_data["yaw_bin_left"] = pd.IntervalIndex(kelmarsh_1_sye_data['yaw_bin']).left

kelmarsh_1_sye_data = kelmarsh_1_sye_data.dropna()

In [ ]:
kelmarsh_1_sye_data

In [ ]:
def cos_curve(x, A, Offset, cos_exp):
    """Computes a cosine exponent curve as a function of yaw misalignment for curve fitting.

    Args:
        x (:obj:`float`): The yaw misalignment input in degrees.
        A (:obj:`float`): The amplitude of the cosine exponent curve.
        Offset (:obj:`float`): The yaw misaligment offset at which the cosine exponent curve is
            maximized in degrees.
        cos_exp (:obj:`float`): The exponent to which the cosine curve is raised.
    Returns:
        :obj:`float`: The value of the cosine exponent curve for the provided yaw misalignment.
    """
    return A * np.cos((np.pi / 180) * (x - Offset)) ** cos_exp

In [ ]:
for ws_bin in set(kelmarsh_1_sye_data["ws_bin_left"]):
    kelmarsh_1_sye_ws_bin = kelmarsh_1_sye_data[kelmarsh_1_sye_data["ws_bin_left"]==ws_bin]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.plot(kelmarsh_1_sye_ws_bin["yaw_bin_left"],kelmarsh_1_sye_ws_bin["Power (kW)"],marker=".",linestyle="")

    curve_fit_params = curve_fit(cos_curve, 
            kelmarsh_1_sye_ws_bin["yaw_bin_left"], 
            kelmarsh_1_sye_ws_bin["Power (kW)"], 
            [kelmarsh_1_sye_ws_bin["Power (kW)"].max(), 0.0, 2.0])[0]

    ax.plot(vane_bins,cos_curve(vane_bins,*curve_fit_params),c="red")
    
    ax.plot(
        2*[curve_fit_params[1]],
        [
                kelmarsh_1_sye_ws_bin["Power (kW)"].max(),
                kelmarsh_1_sye_ws_bin["Power (kW)"].min(),
        ],
        color="red",
        linestyle="--",
        label=rf"Max. Power Vane Angle = {round(curve_fit_params[1],1)}$^\circ$",  # noqa: W605
        )

    plt.title(f"Wind speed {ws_bin}m/s")
    
    ax.legend()
    ax.set_xlim([-25,25])
    ax.set_ylim([kelmarsh_1_sye_ws_bin["Power (kW)"].min(),kelmarsh_1_sye_ws_bin["Power (kW)"].max()])

    

## A slightly better static yaw misalignment analysis (?)

In [ ]:
scada_filtered = scada[(scada["Blade angle (pitch position) A (°)"]<1.5) 
                            & (scada["Blade angle (pitch position) A (°)"]>-1.5)]

In [ ]:
kelmarsh_others = scada_filtered[scada_filtered["Turbine"]!="Kelmarsh_1"]

In [ ]:
site_wind_speed = kelmarsh_others.groupby("Timestamp")["Wind speed (m/s)"].mean()

In [ ]:
kelmarsh_1_filtered["Site wind speed"] = site_wind_speed

In [ ]:
kelmarsh_1_filtered['site_ws_bin'] = pd.cut(kelmarsh_1_filtered['Site wind speed'], ws_bins)

In [ ]:
kelmarsh_1_filtered

In [ ]:
kelmarsh_1_sye_v2 = kelmarsh_1_filtered.groupby(by=["site_ws_bin","yaw_bin"])["Power (kW)"].median()/kelmarsh_1_filtered.groupby(by=["site_ws_bin"])["Power (kW)"].median()
kelmarsh_1_sye_v2 = kelmarsh_1_sye_v2.reset_index()

In [ ]:
kelmarsh_1_sye_v2["ws_bin_left"] = pd.IntervalIndex(kelmarsh_1_sye_v2['site_ws_bin']).left
kelmarsh_1_sye_v2["yaw_bin_left"] = pd.IntervalIndex(kelmarsh_1_sye_v2['yaw_bin']).left

kelmarsh_1_sye_v2 = kelmarsh_1_sye_v2.dropna()

In [ ]:
for ws_bin in set(kelmarsh_1_sye_v2["ws_bin_left"]):
    kelmarsh_1_sye_ws_bin = kelmarsh_1_sye_v2[kelmarsh_1_sye_v2["ws_bin_left"]==ws_bin]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.plot(kelmarsh_1_sye_ws_bin["yaw_bin_left"],kelmarsh_1_sye_ws_bin["Power (kW)"],marker=".",linestyle="")

    curve_fit_params = curve_fit(cos_curve, 
            kelmarsh_1_sye_ws_bin["yaw_bin_left"], 
            kelmarsh_1_sye_ws_bin["Power (kW)"], 
            [kelmarsh_1_sye_ws_bin["Power (kW)"].max(), 0.0, 2.0])[0]

    ax.plot(vane_bins,cos_curve(vane_bins,*curve_fit_params),c="red")
    
    ax.plot(
        2*[curve_fit_params[1]],
        [
                kelmarsh_1_sye_ws_bin["Power (kW)"].max(),
                kelmarsh_1_sye_ws_bin["Power (kW)"].min(),
        ],
        color="red",
        linestyle="--",
        label=rf"Max. Power Vane Angle = {round(curve_fit_params[1],1)}$^\circ$",  # noqa: W605
        )

    plt.title(f"Wind speed {ws_bin}m/s")
    
    ax.legend()
    ax.set_xlim([-25,25])
    ax.set_ylim([kelmarsh_1_sye_ws_bin["Power (kW)"].min(),kelmarsh_1_sye_ws_bin["Power (kW)"].max()])

    